In [ ]:
import pandas as pd
from random import randrange, sample
from numpy.random import uniform

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
s1_path = '/Users/xyuan/Studio/seq_bank/results/2022MarPaper/Sample1.xlsx'
df_s1 = load_data(s1_path)
s2_path = '/Users/xyuan/Studio/seq_bank/results/2022MarPaper/Sample2.xlsx'
df_s2 = load_data(s2_path)

df_s1.shape, df_s2.shape

In [ ]:
# Generate related samples from base samples
df_s1_left, df_s1_right = df_s1.copy(), df_s1.copy()
shift = randrange(-64, 0)
print('shift ', shift)
df_s1_left.Mass += shift

shift = randrange(0, 64)
print('shift ', shift)
df_s1_right.Mass += shift

df_s2_left, df_s2_right = df_s2.copy(), df_s2.copy()
shift = randrange(-64, 0)
print('shift ', shift)
df_s2_left.Mass += shift
shift = randrange(0, 64)
print('shift ', shift)
df_s2_right.Mass += shift

In [ ]:
# Generate related samples, based on df_s1 and df_s2
# shifts = np.sort(uniform(-32, 32, size=100))#.tolist()

def magnify_sample(df, shift_range, times):
    l = list()
    shifts = np.sort(uniform(*shift_range, size=times))
    for shift in shifts:
        df_tmp = df.copy()
        df_tmp.Mass += shift
        df_tmp.RT += uniform(-1, 1)
        l.append(df_tmp)

    return pd.concat(l).reset_index(drop=True)

In [ ]:
df_s2_multi100 = magnify_sample(df_s2, (-32, 32), 100)
df_s2_multi100.head()

In [ ]:
plotly_zones(df_s1_multi100, df_s2_multi100)

In [ ]:
# Generate random samples
size = (100,)
df_rs1 = pd.DataFrame({'Mass': np.sort(uniform(300, 3500, size=size)),
                       'RT': np.sort(uniform(25, 35, size=size))})
df_rs2 = pd.DataFrame({'Mass': np.sort(uniform(300, 3500, size=size)),
                       'RT': np.sort(uniform(20, 30, size=size))})

df_rs1.shape, df_rs2.shape

In [ ]:
# Generate 1/2 random samples
size = (1000,)
df_rs3 = pd.DataFrame({'Mass': np.sort(uniform(300, 3500, size=size)),
                       'RT': np.sort(uniform(25, 35, size=size))})

df_tmp = df_rs3.sample(500)
size = (500,)
df_rs4 = pd.DataFrame({'Mass': np.sort(uniform(300, 3500, size=size)),
                       'RT': np.sort(uniform(20, 30, size=size))})
df_rs4 = pd.concat([df_rs4, df_tmp]).reset_index(drop=True)

df_rs3.shape, df_rs4.shape

In [ ]:
size = (1000,)
df_rs5 = pd.DataFrame({'Mass': np.sort(uniform(300, 3500, size=size)),
                       'RT': np.sort(uniform(25, 35, size=size))})
df_rs6 = df_rs5.copy()

In [ ]:
# dfm_tmp = match_dfs(df_rs3, df_rs4)
# dfm_tmp.shape

In [ ]:
df_rs1_left, df_rs1_right = df_rs1.copy(), df_rs1.copy()
shift = randrange(-64, 0)
print('shift ', shift)
df_rs1_left.Mass += shift
shift = randrange(0, 64)
print('shift ', shift)
df_rs1_right.Mass += shift

df_rs2_left, df_rs2_right = df_rs2.copy(), df_rs2.copy()
shift = randrange(-64, 0)
print('shift ', shift)
df_rs2_left.Mass += shift
shift = randrange(0, 64)
print('shift ', shift)
df_rs2_right.Mass += shift

In [ ]:
# plotly_zones(df_s1, df_s2)
# plotly_zones(df_s1_left, df_s1_right)
# plotly_zones(df_s2_left, df_s2_right)
# plotly_zones(df_rs1, df_rs2)
# plotly_zones(df_rs3, df_rs4)
plotly_zones(df_rs1, df_rs2)

In [ ]:
# sample_size = 200
# dfa, dfb, mass_range = df_rs1.sample(sample_size).copy(), df_rs2.sample(sample_size).copy(), np.arange(-64, 64, 0.1)
# dfa, dfb, mass_range = df_s1.copy(), df_s2.copy(), np.arange(-64, 64, 0.1)
dfa, dfb, mass_range = df_s1_multi100.copy(), df_s2_multi100.copy(), np.arange(-64, 64, 0.1)


import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
params = [(dfa, dfb, shift) for shift in mass_range]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)

In [ ]:
fig = px.line(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:

def func(dfa, dfb, shift):
    dfm = match_dfs(dfa, dfb, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

def hit_map(dfa, dfb, mass_range):

    PROCESSES = 8
    params = [(dfa, dfb, shift) for shift in mass_range]
    with multiprocessing.Pool(PROCESSES) as pool:
        shift_hits = pool.starmap(func, params)

    return pd.DataFrame(shift_hits)

mass_range = np.arange(-64, 64, 0.1)
sample_pairs = [
#                 (df_s1, df_s1, mass_range),
#                  (df_s1, df_s1_left, mass_range),
#                  (df_s1, df_s1_right, mass_range),
                 
#                  (df_s2, df_s2, mass_range),
#                  (df_s2, df_s2_left, mass_range),
#                  (df_s2, df_s2_right, mass_range),
                
#                  (df_rs1, df_rs1, mass_range),
#                 (df_rs1, df_rs1_left, mass_range),
#                 (df_rs1, df_rs1_right, mass_range),
                
#                 (df_rs2, df_rs2, mass_range),
#                 (df_rs2, df_rs2_left, mass_range),
#                 (df_rs2, df_rs2_right, mass_range),
                 
#                  (df_s1, df_s2, mass_range),
#                 (df_s1, df_rs1, mass_range),
#                 (df_s2, df_rs1, mass_range),
#                 (df_rs1, df_rs2, mass_range),
#                 (df_rs3, df_rs4, mass_range),
#                 (df_rs5, df_rs6, mass_range),
#                 (df_s1_multi100, df_s1_multi100, mass_range),
                (df_s1_multi100, df_s2_multi100, mass_range),
                ]

hitmap_list = [hit_map(*pair) for pair in sample_pairs]


In [ ]:
hm_list = list()
for idx, df in enumerate(hitmap_list):
    df_hm = df.copy()
    df_hm['idx'] = idx
    hm_list.append(df_hm)

df_hms = pd.concat(hm_list)
fig = px.line(df_hms[df_hms.hit<500], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'}, color='idx')
fig.show()
play_beep()

In [ ]:
# save data
# with pd.ExcelWriter('data/random_two_1k.xlsx') as writer:
#     df_rs1.to_excel(writer, sheet_name='RandomSample1')
#     df_rs2.to_excel(writer, sheet_name='RandomSample2')
#     hitmap_list[0].to_excel(writer, sheet_name='hitmap')

# with pd.ExcelWriter('data/random_1k_and_500.xlsx') as writer:
#     df_rs3.to_excel(writer, sheet_name='RandomSample3')
#     df_rs4.to_excel(writer, sheet_name='RandomSample4')
#     hitmap_list[0].to_excel(writer, sheet_name='hitmap')

# with pd.ExcelWriter('data/random_1k_and_duplicated_1k.xlsx') as writer:
#     df_rs5.to_excel(writer, sheet_name='RandomSample5')
#     df_rs6.to_excel(writer, sheet_name='RandomSample6')
#     hitmap_list[0].to_excel(writer, sheet_name='hitmap')

# with pd.ExcelWriter('data/s1_1k_and_duplicated_1k.xlsx') as writer:
#     df_rs5.to_excel(writer, sheet_name='RandomSample5')
#     df_rs6.to_excel(writer, sheet_name='RandomSample6')
#     hitmap_list[0].to_excel(writer, sheet_name='hitmap')

# with pd.ExcelWriter('data/s1_sample500_and_s2_sample500.xlsx') as writer:
#     dfa.to_excel(writer, sheet_name='S1_sample500')
#     dfb.to_excel(writer, sheet_name='S2_sample500')
#     df_shift_hits.to_excel(writer, sheet_name='hitmap')

# with pd.ExcelWriter('data/s1_sample100_and_s2_sample100.xlsx') as writer:
#     dfa.to_excel(writer, sheet_name='S1_sample100')
#     dfb.to_excel(writer, sheet_name='S2_sample100')
#     df_shift_hits.to_excel(writer, sheet_name='hitmap')

with pd.ExcelWriter('data/s1_sample200_and_s2_sample200.xlsx') as writer:
    dfa.to_excel(writer, sheet_name='S1_sample200')
    dfb.to_excel(writer, sheet_name='S2_sample200')
    df_shift_hits.to_excel(writer, sheet_name='hitmap')

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))
# df_plt = df_sh_top.copy().sort_values('shift')
# df_plt1 = df_plt.copy()
# df_plt1['shift'] = df_plt1['shift'].astype(int)
# df_plt1 = df_plt1.set_index('shift')
# df_plt1.plot(kind='bar',figsize=(2.5, 1.5))

hitmap_list[0]['hit'].plot.line()
# fg = px.line(df_hms, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'}, color='idx')
# fg.show()
# plt.legend()

fig.tight_layout()
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
plt.savefig(svg_fpath, transparent=True, dpi=300)